In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as trans
import matplotlib
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [35]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [36]:
train_set = dsets.MNIST(
    root='./data', 
    train=True, 
    transform=trans.ToTensor(), 
    download=True)

In [37]:
print('Number of images {}'.format(len(train_set)))
print('Type {}'.format(type(train_set[0][0])))
print('size of images {}'.format(train_set[0][0].size()))

Number of images 60000
Type <class 'torch.Tensor'>
size of images torch.Size([1, 28, 28])


In [38]:
class MultiLogisticModel(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(MultiLogisticModel, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        
    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        return out

In [57]:
in_dim = 28*28
out_dim = 10
model = MultiLogisticModel(in_dim,out_dim).cuda()
model.to(device)

lr = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda x: x*0.7)


loss_function = nn.CrossEntropyLoss()

In [58]:
batchSize = 100
trainloader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=batchSize,
    num_workers = 12,
    pin_memory=True,
    shuffle=True)

In [66]:
epochs = 100
for epoch in range(epochs):
    runningLoss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        optimizer.zero_grad()
        images = images.view(-1, 28 * 28)
        images = images.to(device)
        labels = labels.to(device)
        out = model(images).to(device)
        loss = loss_function(out, labels)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
    print('epoch {}, loss {}'.format(epoch, runningLoss))
    lr_scheduler.step()
    if epoch%5 == 0:
        lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda x: x*0.7)

epoch 0, loss 923.2346438169479
epoch 1, loss 923.2074882984161
epoch 2, loss 923.2074898481369
epoch 3, loss 923.2165929079056
epoch 4, loss 923.2242610454559
epoch 5, loss 923.2310808897018
epoch 6, loss 923.203705906868
epoch 7, loss 923.2037031650543
epoch 8, loss 923.2124952077866
epoch 9, loss 923.2196451425552
epoch 10, loss 923.2262853384018
epoch 11, loss 923.1990579366684
epoch 12, loss 923.1990647315979
epoch 13, loss 923.2079836130142
epoch 14, loss 923.215115070343
epoch 15, loss 923.2220945358276
epoch 16, loss 923.195160150528
epoch 17, loss 923.195161819458
epoch 18, loss 923.2037670612335
epoch 19, loss 923.209562420845
epoch 20, loss 923.2184315919876
epoch 21, loss 923.1903346776962
epoch 22, loss 923.1903326511383
epoch 23, loss 923.1992877721786
epoch 24, loss 923.2060656547546
epoch 25, loss 923.2117710113525
epoch 26, loss 923.1871325969696
epoch 27, loss 923.1871324777603
epoch 28, loss 923.1954009532928
epoch 29, loss 923.2022411823273
epoch 30, loss 923.209189

In [18]:
import numpy as np
def successRate(predicted, labels):
    predict = [np.argmax(p.detach().numpy()) for p in predicted]
    actual =  [labels[i].item() for i in range(len(predicted))]
    correct = [i for i, j in zip(predict, actual) if i == j]
    return(len(correct)/(len(predict)))

predicted = model.forward(images) 
predicted.size()
successRate(predicted, labels)

0.82

In [20]:
testSet = dsets.MNIST(root='./data', train = False, transform=trans.ToTensor(), download=True)
testloader = torch.utils.data.DataLoader(dataset=testSet,  batch_size = 10000, shuffle = True)

testData = iter(testloader)
images, labels = testData.next()
output = model.forward(images.view(-1, 28 * 28))
successRate(output,labels)

0.8383

In [17]:
torch.save(model.state_dict(), 'multiclassmodel.pkl')
model.load_state_dict(torch.load('multiclassmodel.pkl'))